# Exercise 2.03: Preparing the Quandl Data for Prediction

The goal of this exercise is to download an external dataset from the Quandl library and then prepare it so that it is ready for use in our linear regression models.

The following steps will help you to complete this exercise:

  > **Note**  
  > You need to install Qaundl library

1.- Download the data into a DataFrame using Quandl for the S&P 500 between 1950 and 2019. Its ticker is `“YALE/SPCOMP”`:

In [15]:
import quandl
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection
data_frame = quandl.get('../syllabus_intelligencesystems/data/YALE_SPCOMP_test.csv')
data_frame.head()

ValueError: The column index must be expressed as an integer for ../syllabus_intelligencesystems/data/YALE_SPCOMP_test.csv.

2.- Use the `head()` method to visualize the columns inside the `data_frame.head()` DataFrame.

Output:

![Figure 2.18](img/fig2_18.jpg)

In [ ]:
data_frame = data_frame[['Long Interest Rate', 'Real Price', 'Real Dividend']]
data_frame

A few features seem to highly correlate with each other. For instance, the `Real Dividend` column grows proportionally with `Real Price`. The ratio between them is not always similar, but they do correlate.

As regression is not about detecting the correlation between features, we would rather get rid of the features that we know are correlated and perform regression on the features that are non-correlated. In this case, we will keep the `Long Interest Rate`, `Real Price`, and `Real Dividend` columns.

3.- Keep only the relevant columns in the `Long Interest Rate`, `Real Price`, and `Real Dividend` DataFrames:

Output:

![Figure 2.19](img/fig2_19.jpg)

In [ ]:
data_frame.fillna(method='ffill', inplace=True)
data_frame

You can see that the DataFrame contains a few missing values `NaN`. As regression doesn't work with missing values, we need to either replace them or delete them. In the real world, we will usually choose to replace them. In this case, we will replace the missing values by the preceding values using a method called **forward filling**.

4.- Replace the missing values with a forward filling.

Output:

![Figure 2.20](img/fig2_20.jpg)

In [ ]:
data_frame['Real Price Label'] = data_frame['Real Price'].shift(-3)
data_frame

Now that we have cleaned the missing data, we need to create our label. We want to predict the `Real Price` column 3 months in advance using the current `Real Price`, `Long Interest` Rate, and `Real Dividend` columns. In order to create our label, we need to shift the `Real Price` values up by three units and call it `Real Price Label`.

6.- Create the `Real Price Label` label by shifting `Real Price` by 3 months.

Output:

![Figure 2.21](img/fig2_21.jpg)

In [16]:
features = np.array(data_frame.drop('Real Price Label', 1))
scaled_features = preprocessing.scale(features)
scaled_features

AttributeError: 'str' object has no attribute 'drop'

The side effect of shifting these values is that missing values will appear in the last three rows for `Real Price Label`, so we need to remove the last three rows of data. However, before that, we need to convert the features into a NumPy array and scale it. We can use the `drop` method of the DataFrame to remove the label column and the preprocessing function from `sklearn` to scale the features.

6.- Create a NumPy array for the features and scale it.

Output:

```
array([[-1.14834808, -1.13030753, -1.19145337],
       [-1.14109376, -1.12504171, -1.17960932],
       [-1.13383944, -1.12398084, -1.17363711],
       ...,
       [-1.36075461,  2.93924962,  3.67194058],
       [-1.32593387,  3.12706161,  3.7120735 ],
       [-1.3085235 ,  3.23602381,  3.75460521]])
```

In [ ]:
scaled_features_latest_3 = scaled_features[-3:]
scaled_features = scaled_features[:-3]
scaled_features

The `1 in the second argument specifies that we are dropping columns. As the original DataFrame was not modified, the label can be directly extracted from it. Now that the features are scaled, we need to remove the last three values of the features as they are the features of the missing values in the label column. We will save them for later in the prediction part.

7.- Remove the last three values of the features array and save them into another array

Output:

```
array([[-1.14834808, -1.13030753, -1.19145337],
       [-1.14109376, -1.12504171, -1.17960932],
       [-1.13383944, -1.12398084, -1.17363711],
       ...,
       [-1.07522452,  2.86136628,  3.47166907],
       [-1.12049149,  2.78043332,  3.50007195],
       [-1.23888201,  2.83151407,  3.54006023]])
```

In [ ]:
data_frame.dropna(inplace=True)
data_frame

The `scaled_features` variable doesn't contain the three data points anymore as they are now in `scaled_features_latest_3`. Now we can remove the last three rows with missing data from the DataFrame, then convert the label into a NumPy array using `sklearn`.

8.- Remove the rows with missing data

Output:

![Figure 2.22](img/fig2_22.jpg)

In [ ]:
label = np.array(data_frame['Real Price Label'])
label

9.- Now let's see if we have accurately created our label.

Output:

![Figure 2.23](img/fig2_23.jpg)

Our variable contains all the labels and is exactly the same as the `Real Price Label` column in the DataFrame.

Our next task is to separate the training and testing data from each other. As we saw in the Splitting Data into Training and Testing section, we will use $90\%$ of the data as the training data and the remaining $10\%$ as the test data.

10.- Split the `features` data into training and test sets using `sklearn`.

In [ ]:
from sklearn import model_selection
(features_train, features_test, label_train, label_test) = model_selection.train_test_split(scaled_features, label, test_size=0.1, random_state=8)

By completing this exercise, we have learned all the required steps for data preparation before performing a regression.